# KOR_QPAIR 학습
* 학습 데이터 : kor_qpair
* 대상 모델 : KoELECTRA

In [93]:
## 관련 패키지 설치
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from transformers import TrainingArguments
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [95]:
# data set down load
dataset = load_dataset("kor_qpair")

  0%|          | 0/3 [00:00<?, ?it/s]

In [96]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'is_duplicate'],
        num_rows: 6136
    })
    test: Dataset({
        features: ['question1', 'question2', 'is_duplicate'],
        num_rows: 758
    })
    validation: Dataset({
        features: ['question1', 'question2', 'is_duplicate'],
        num_rows: 682
    })
})

In [97]:
dataset['train'][0]

{'question1': '여자친구가 데이트 시간을 너무 안 지켜.',
 'question2': '여자친구가 데이트 시간을 너무 안 지켜.',
 'is_duplicate': 0}

In [98]:
pd.DataFrame.from_dict(dataset['train'])

,question1,question2,is_duplicate
0,여자친구가 데이트 시간을 너무 안 지켜.,여자친구가 데이트 시간을 너무 안 지켜.,0
1,집앞에서 첫사랑 만남,집앞에서 첫사랑 만남,0
2,어이가 없네,어이상실,0
3,전화 안 받으니까 초조해져,전화 안 받는 이유가 뭘까,0
4,연인인데 정치견해가 달라,연인인데 정치적인 견해가 달라,0
...,...,...,...
6131,전남친 마주쳤어,전남친이 만나자고 합니다.,1
6132,재회하면 또 그렇겠지,저 잘 잊을 수 있겠죠?,1
6133,혼자인게 좋다,혼자인게 차라리 편해,1
6134,짧은연애인데 미련때문에 연락을 할지말지 ㅠ.ㅠ,찌질한 이별중,1


In [99]:
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')  # Hugging face에서 모델 구조를 잘 모를 경우 모든 모델에서 사용 할 수 있다.
model = AutoModelForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator')

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [100]:
text = dataset['train'][0]['question1']

tokenizer(text, padding='max_length', max_length=50)

{'input_ids': [2, 6555, 4638, 4050, 4070, 11523, 6251, 4292, 6395, 3081, 7275, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [101]:
tokenizer.tokenize(text)

['여자', '##친', '##구', '##가', '데이트', '시간', '##을', '너무', '안', '지켜', '.']

In [102]:
tokenizer(dataset['train'][0]['question1'], dataset['train'][0]['question2'])

{'input_ids': [2, 6555, 4638, 4050, 4070, 11523, 6251, 4292, 6395, 3081, 7275, 18, 3, 6555, 4638, 4050, 4070, 11523, 6251, 4292, 6395, 3081, 7275, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [103]:
dataset = dataset.rename_column('is_duplicate', 'label')

In [104]:
training_args = TrainingArguments('230429_kor_qpair_electra')

In [105]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignor

In [106]:
def tokenizer_function(example):
  return tokenizer(example['question1'], example['question2'], truncation=True)

tokenized_dataset = dataset.map(tokenizer_function, batched=True)

Map:   0%|          | 0/682 [00:00<?, ? examples/s]

In [107]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [108]:
from transformers import Trainer

In [109]:
trainer = Trainer(
  model,
  training_args,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['test'],
  data_collator=data_collator,
  tokenizer=tokenizer
  )

In [110]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.310000
1000,0.200200
1500,0.148500
2000,0.064500


TrainOutput(global_step=2301, training_loss=0.16728660968115722, metrics={'train_runtime': 232.8354, 'train_samples_per_second': 79.06, 'train_steps_per_second': 9.883, 'total_flos': 281615162550240.0, 'train_loss': 0.16728660968115722, 'epoch': 3.0})

In [111]:
predictons = trainer.predict(test_dataset=tokenized_dataset['test'])

In [112]:
predictons

PredictionOutput(predictions=array([[-3.910741 ,  3.3917375],
       [-3.57591  ,  3.0183809],
       [ 3.7533207, -3.3270671],
       ...,
       [-4.000247 ,  3.4989998],
       [ 3.7690153, -3.351798 ],
       [ 3.744079 , -3.3389904]], dtype=float32), label_ids=array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
    

In [113]:
label_ids = predictons.label_ids
predictions_list = [np.argmax(item) for item in predictons.predictions]

In [114]:
label_ids[:10]

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [115]:
print(classification_report(y_true=label_ids, y_pred=predictions_list))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       508
           1       0.96      0.89      0.92       250

    accuracy                           0.95       758
   macro avg       0.95      0.93      0.94       758
weighted avg       0.95      0.95      0.95       758



## Evaluation
코드는 키워드와 입력된 데이터를 받아 비교하는 형태로 되어 있으나... 키워드를 vector화 하여 저장하고 입력된 데이터를 vector화 하여 비교하는 것이 빠르다.

In [116]:
import torch

In [117]:
device = torch.device("cuda")
question_answer = [['11기 수강생 몇 명인가요?', '11명입니다.'], ['자연어처리 재미있나요?', '엄청재미잇습니다. ㅎㅎ']]
user_question = '11기 수업듣는 수강생은 몇 명인가요?'
tokenized_text = tokenizer('11기 수강생 몇 명인가요?', user_question, return_tensors='pt').to(device)

In [119]:
model.eval()
output = model(**tokenized_text)

In [120]:
np.argmax(output.logits.cpu().detach()).tolist()

0

In [121]:
tokenized_text = tokenizer('자연어처리 재미있나요?', user_question, return_tensors='pt').to(device)
output = model(**tokenized_text)
np.argmax(output.logits.cpu().detach()).tolist()

1